In [1]:
# системные модули
import os
import warnings
import sys
from typing import Union
# работа с данными
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
# визуализация
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

In [2]:
dates = ['FLTDAT', 'DAIS']
dtypes = {'FLTNUM': 'int16', 'TARIF_RUB': 'float16'}
bb = pd.read_csv('F:\\SVO-AER_2018-01-01--2019-12-31BB.csv',
                 delimiter=';', dtype=dtypes, parse_dates=dates)
bb = bb.dropna(how='any', axis=0)
bb.drop_duplicates(inplace=True)
bb['DEPARTURE'] = bb['FLTDAT'] + pd.to_timedelta(bb['FLTTIME'])
bb = bb[['FLTDAT', 'DEPARTURE', 'FLTNUM']].drop_duplicates()
bb.head()

,FLTDAT,DEPARTURE,FLTNUM
0,2018-01-21,2018-01-21 13:35:00,1124
1,2018-01-10,2018-01-10 09:10:00,1122
2,2018-01-11,2018-01-11 09:10:00,1122
3,2018-01-26,2018-01-26 17:35:00,1140
4,2018-01-06,2018-01-06 09:10:00,1122


In [3]:
%%time
dates = ['SDAT_S', 'DD']
dtypes = {'FLT_NUM': 'int16', 'FCLCLD': 'int16', 'PASS_BK': 'int16','SA': 'int16',
          'AU': 'int16', 'PASS_DEP': 'int16', 'NS': 'int16', 'DTD': 'int16'}
hh = pd.read_csv('F:\\hh_for_profiles.csv', dtype=dtypes, parse_dates=dates)

Wall time: 1min


In [74]:
def get_flight(fltnum: int, date: datetime, flttime: pd.Timedelta) -> (str, pd.DataFrame):
    """Получение данных рейса для восстановления спроса"""
    departure = date + flttime
    data = bb[(bb['FLTNUM'] == fltnum) & (bb['DEPARTURE'] == departure)]
    print(f"Рейс:", fltnum, departure)
    data = pd.merge(data, hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'])
    data = data.dropna(how='any', axis=0)
    data.drop(columns=['FLTDAT', 'FLT_NUM', 'DD'], inplace=True)
    data.drop_duplicates(inplace=True)
    print(f"Данных: {flight.shape[0]:_}")
    return f"{fltnum}, {departure}", data

def fill_missed_values(data: pd.DataFrame, code: str) -> pd.DataFrame:
    """Заполнение пропущенных значений в DTD"""
    flight_class = data[data['SEG_CLASS_CODE'] == code]
#     max_dtd = range(data['DTD'].max(), -1 - 1, -1)
    max_dtd = range(300, -1 - 1, -1)
    flight_class = flight_class.set_index('DTD').sort_values(by='DTD', ascending=False)
    flight_class = flight_class.reindex(max_dtd).ffill().reset_index()
    return flight_class

def get_regression(data: pd.DataFrame) -> (Union[np.float64, np.int16], Union[np.float64, np.int16], int):
    """Получение коэффициентов линейной регрессии"""
    # получение первого дня последнего цензурирования
    first_last_one = np.where(data['FCLCLD'] == 0)[0]
    if len(first_last_one) != 0:
        first_last_one = first_last_one[-1] + 1
        # первый день последнего цензурирования влияет на наклон регрессии
        data.loc[data['DTD'] == first_last_one, 'FCLCLD'] = -1
    # все дни продаж билетов:
    ts = max(data['DTD'].unique())
    # если время продаж билетов < 20%:
    if len(data[data['FCLCLD'] == 0]) / len(data) < .2:
        print("время продаж билетов < 20%", 0, 0, ts, end=' | ')
        return 0, 0, ts
    # фильтрация по времени нецензурирования:
    data = data[data['FCLCLD'] != 1]
    # числа для МНК:
    z = data['PASS_BK'].values
    z = z[~np.isnan(z)]
    t = data['DTD'].values[:z.shape[0]].reshape(-1, 1)
    # если после фильтра осталась 1 или 0 точек (линейная регрессия не существует):
    if z.size <= 1:
        print("линейная регрессия не существует", 0, 0, ts, end=' | ')
        return 0, 0, ts
    # коэффициенты линейной регрессии:
    reg = LinearRegression().fit(t, z)
    b1, b0 = reg.coef_[0], reg.intercept_
    print("ok", b1, b0, ts, end=' | ')
    return b1, b0, ts

def get_ts_streak(b1: np.float64, b0: np.float64, ts: np.int16) -> Union[np.float64, np.int16]:
    """Получение значения Т штрих"""
    if b1 <= 0:
        print("B1 <= 0", ts)
        return ts  # max(DTD)
    ts_streak = -b0 / b1
    if ts_streak < 0 or ts_streak > ts:
        print("Ts_streak < 0 or Ts_streak > Ts", ts)
        return ts  # max(DTD)
    print("ok", ts_streak)
    return ts_streak

def get_restored_demand(ts_streak: Union[np.float64, np.int16], data: pd.DataFrame, class_code: str):
    """Получение восстановленного спроса"""
    tmax = 10
    flight_class = data[data['SEG_CLASS_CODE'] == class_code]
    flight_class = flight_class[['DTD', 'PASS_BK', 'PASS_DEP', 'FCLCLD']]
    flight_class = flight_class.sort_values(by=['DTD'], ascending=False).reset_index(drop=True)
    # первый день последнего раза закрытия продаж
    first_last_one = np.where(flight_class['FCLCLD'] == 0)[0]
    # случай, если продажи не были закрыты в 0 или -1 день (спрос уже восстановленный):
    restored_demand_fail = flight_class[flight_class['DTD'].isin([-1, 0])]
    if len(first_last_one) == 0:
        restored_demand_fail['restored'] = restored_demand_fail['PASS_BK']
        return restored_demand_fail
    # Если больше трети было последнее цензурирование
    if len(first_last_one) / len(flight_class) > .33:
        ts_streak = flight_class['DTD'].max()
    first_last_one = first_last_one[-1] + 1
    # получения последних дней цензурирования
    restored_demand = flight_class[first_last_one:].reset_index(drop=True)
    # восстановление проходит только на последние Tmax дней от 0
    restored_demand = restored_demand[restored_demand['DTD'] <= tmax]
    # случай, если продажи не были закрыты даже в -1 день (спрос уже восстановленный):
    if len(restored_demand) == 0:
        restored_demand_fail['restored'] = restored_demand_fail['PASS_BK']
        return restored_demand_fail
    # количество дней последнего цензурирования до вылета:
    Tc = max(restored_demand['DTD'].unique())
    # бронирования класса на момент последнего закрытия продаж:
    Zc = restored_demand[restored_demand['DTD'] == Tc]['PASS_BK']
    restored_demand['restored'] = restored_demand.apply(
        lambda x: (Zc * (ts_streak - x['DTD']) ** 2) /
                  (     (ts_streak - Tc) ** 2)
        , axis=1
    )
    restored_demand.rename_axis(class_code, axis="columns", inplace=True)
    return restored_demand

def get_ful_restoration(data: pd.DataFrame) -> pd.DataFrame:
    """
    Получение восстановленного спроса по всем классам вылета в табличном виде на каждый день DTD,
    где в -1 день отображается фактическое кол-во пассажиров, с 0 до TMAX - восстановленный спрос,
    с TMAX + 1 - кол-во фактических бронирований.
    """
    full_restored = dict()
    for class_code in flight['SEG_CLASS_CODE'].unique():
        print(class_code, end=' | ')
        flight_class = fill_missed_values(data, class_code)
        flight_class.to_excel('test.xlsx')
        b1, b0, ts = get_regression(flight_class)
        ts_streak = get_ts_streak(b1, b0, ts)
        flight_class = get_restored_demand(ts_streak, flight, class_code)
        flight_cp = data[data['SEG_CLASS_CODE'] == class_code][['PASS_BK', 'DTD']]
        # фактическое кол-во вылетевших людей
        pass_dp = flight_class[flight_class['DTD'] == -1]['PASS_DEP'].values[0]
        # восстановленный спрос
        pass_bk_restored = flight_class[flight_class['DTD'] != -1]
        max_dtd = pass_bk_restored['DTD'].max()
        # получение восстановленного спроса с 0 до Tmax
        pass_bk_restored = pass_bk_restored['restored'].values.tolist()[::-1]
        full_restored.setdefault(class_code, [pass_dp]).extend(pass_bk_restored)
        # фактическое количество бронирования с Tmax + 1 дня до Ts
        pass_bk_unrestored = flight_cp.loc[flight_cp['DTD'] > max_dtd]['PASS_BK'].values.tolist()[::-1]
        full_restored[class_code].extend(pass_bk_unrestored)
    return pd.DataFrame(full_restored)

def class_graph(restored_table: pd.DataFrame) -> None:
    """График продаж по классам"""
    restored_table = restored_table.transpose().reset_index()
    restored_table = restored_table.melt(id_vars='index', var_name='x', value_name='y')
    restored_table['x'] = -restored_table['x']
    fig = px.scatter(restored_table, x='x', y='y', color='index',
                     title=f'Восстановление спроса для {flight_name}')
    fig.update_layout(xaxis_title="День до вылета", yaxis_title="Восстановленный спрос")
    fig.update_xaxes(tickvals=list(range(restored_table['x'].min(), 2, 14)))
    fig.show()

In [5]:
%%time
flight_name, flight = get_flight(1122,
                                 datetime.strptime('2018-01-06', '%Y-%m-%d'),
                                 pd.Timedelta('09:10:00'))

Рейс: 1122 2018-01-06 09:10:00
Данных: 4_158
Wall time: 1.88 s


In [47]:
restored = get_ful_restoration(flight)
restored

,J,C,D,I,Z,O,Y,B,M,U,...,H,L,X,Q,T,E,N,R,G,V
-1,0,0,3,0,0.0,6.000000,0,0,0.0,0.0,...,5.00000,5.00000,13.000000,4.000000,22.000000,22.000000,83.000000,3.000000,0,0
0,0,0,3,0,0.0,6.055944,0,0,0.0,0.0,...,5.04662,5.04662,14.262905,4.075116,26.737552,23.055008,93.640832,3.298332,0,0
1,0,0,3,0,0.0,6.000000,0,1,0.0,0.0,...,5.00000,5.00000,14.131147,4.037471,26.490555,22.842030,92.775794,3.267862,0,0
2,0,0,3,0,0.0,6.000000,0,0,0.0,0.0,...,4.00000,2.00000,14.000000,4.000000,26.244705,22.630040,91.914770,3.237534,0,0
3,2,0,3,0,0.0,4.000000,0,0,0.0,0.0,...,4.00000,2.00000,8.000000,2.000000,26.000000,22.419038,91.057761,3.207348,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
195,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
202,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
209,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0


In [75]:
class_graph(restored)